In [2]:
import numpy as np
import pandas as pd
from doubleml.datasets import fetch_bonus
from doubleml import DoubleMLData
import statsmodels.api as sm
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LassoCV
from doubleml import DoubleMLPLR   
from doubleml import DoubleMLPLR  ,DoubleMLPLIV         
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
from sklearn.linear_model import LinearRegression, LogisticRegression
from doubleml.datasets import fetch_401K
import wooldridge

In [ ]:
c1985 <- subset(CigarettesSW, year == "1985")

# define differences in variables
packsdiff <- log(c1995$packs) - log(c1985$packs)

pricediff <- log(c1995$price/c1995$cpi) - log(c1985$price/c1985$cpi)

incomediff <- log(c1995$income/c1995$population/c1995$cpi) -
log(c1985$income/c1985$population/c1985$cpi)

salestaxdiff <- (c1995$taxs - c1995$tax)/c1995$cpi - (c1985$taxs - c1985$tax)/c1985$cpi

cigtaxdiff <- c1995$tax/c1995$cpi - c1985$tax/c1985$cpi

In [4]:
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/CigarettesSW.csv')
df.head()

,Unnamed: 0,state,year,cpi,population,packs,income,tax,price,taxs
0,1,AL,1985,1.076,3973000.0,116.486282,46014968,32.500004,102.181671,33.348335
1,2,AR,1985,1.076,2327000.0,128.534592,26210736,37.000000,101.474998,37.000000
2,3,AZ,1985,1.076,3184000.0,104.522614,43956936,31.000000,108.578751,36.170418
3,4,CA,1985,1.076,26444000.0,100.363037,447102816,26.000000,107.837341,32.104000
4,5,CO,1985,1.076,3209000.0,112.963539,49466672,31.000000,94.266663,31.000000


In [16]:
df.dtypes

Unnamed: 0      int64
state          object
year            int64
cpi           float64
population    float64
packs         float64
income          int64
tax           float64
price         float64
taxs          float64
dtype: object

In [21]:
df85 = df[df.year==1985]
df95 = df[df.year==1995]
df95.shape

(48, 10)

In [13]:
packsdiff = np.log(df95.packs) - np.log(df85.packs)
pricediff = np.log(df95.price/df95.cpi)- np.log(df85.price/df85.cpi)
incomediff = np.log(df95.income/df95.population/df95.cpi)- np.log(df85.income/df85.population/df85.cpi)
salestaxdiff = (df95.taxs-df95.tax)/df95.cpi - (df85.taxs-df85.tax)/df85.cpi
cigtaxdiff = df95.tax/df95.cpi - df85.tax/df85.cpi

pd.DataFrame(np.c_[packsdiff, pricediff, incomediff, salestaxdiff, cigtaxdiff], columns = ['pack', 'price', 'inc', 'sales', 'cig'])

,pack,price,inc,sales,cig
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
91,NaN,NaN,NaN,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN


SyntaxError: cannot assign to attribute here. Maybe you meant '==' instead of '='? (2118269489.py, line 1)

In [14]:
df = df[['lwage','nearc2', 'nearc4' ,'age', 'married', 'smsa', 'educ' ]].dropna()

In [15]:
np.random.seed(1)
df.head(5)
y = df['lwage']
d = df['educ']
x = df[['age', 'married', 'smsa']]
z = df[['nearc2', 'nearc4']]

l = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
g = clone(l)
m = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
r = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)

def score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

print(y.shape, x.shape, d.shape, z.shape)

(3003,) (3003, 3) (3003,) (3003, 2)


In [27]:
import statsmodels.api as sm
OLS = sm.OLS(y, sm.add_constant(np.c_[d, x])).fit()

In [17]:
from statsmodels.sandbox.regression.gmm import IV2SLS
endog = df['lwage']
exog =  df[['educ', 'age', 'married', 'smsa']]
instr = df[['nearc2', 'nearc4' ,'age', 'married', 'smsa']]
exog_constant = sm.add_constant(exog)
instr_constant = sm.add_constant(instr)
IV2SLS = IV2SLS(endog, exog_constant, instrument = instr_constant).fit()
IV2SLS.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                  lwage   R-squared:                      -0.093
Model:                         IV2SLS   Adj. R-squared:                 -0.094
Method:                     Two Stage   F-statistic:                     118.4
                        Least Squares   Prob (F-statistic):           7.21e-94
Date:                Tue, 13 Dec 2022                                         
Time:                        12:19:59                                         
No. Observations:                3003                                         
Df Residuals:                    2998                                         
Df Model:                           4                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.3827      0.465      7.275      0.000       2.471       4.294
educ           0.1473      0.038      3.908      0.000       0.073       0.221
age            0.0345      0.003     12.389      0.000       0.029       0.040
married       -0.0479      0.005     -9.454      0.000      -0.058      -0.038
smsa           0.0900      0.045      1.998      0.046       0.002       0.178
==============================================================================
Omnibus:                       20.554   Durbin-Watson:                   1.849
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.283
Skew:                          -0.146   Prob(JB):                     8.79e-06
Kurtosis:                       3.318   Cond. No.                         337.
==============================================================================
"""

In [23]:
data = DoubleMLData(df.sample(int(df.shape[0]*0.8)), 'lwage', 'educ', z_cols=['nearc2', 'nearc4'])
SML = DoubleMLPLIV(data, l,m,g,r, n_folds=1,apply_cross_fitting=False,score=score)
#SML.fit();

/usr/local/lib/python3.10/site-packages/doubleml/double_ml.py:953: UserWarning: Learner provided for ml_m is probably invalid: RandomForestClassifier(max_depth=5, n_estimators=500) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


In [25]:
data = DoubleMLData(df.sample(int(df.shape[0]*0.8)), 'lwage', 'educ', z_cols=['nearc2', 'nearc4'])
OML = DoubleMLPLIV(data,l,m,g,r, n_folds=1,apply_cross_fitting=False)
#OML.fit();

/usr/local/lib/python3.10/site-packages/doubleml/double_ml.py:953: UserWarning: Learner provided for ml_m is probably invalid: RandomForestClassifier(max_depth=5, n_estimators=500) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')
/usr/local/lib/python3.10/site-packages/doubleml/double_ml_pliv.py:153: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(('A learner ml_g has been provided for score = "partialling out" but will be ignored. "'


In [26]:
data = DoubleMLData(df.sample(int(df.shape[0]*1)), 'lwage', 'educ', z_cols=['nearc2', 'nearc4'])
DML = DoubleMLPLIV(data, l,m,g,r, n_folds=5,apply_cross_fitting=True)
DML.fit();

/usr/local/lib/python3.10/site-packages/doubleml/double_ml.py:953: UserWarning: Learner provided for ml_m is probably invalid: RandomForestClassifier(max_depth=5, n_estimators=500) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')
/usr/local/lib/python3.10/site-packages/doubleml/double_ml_pliv.py:153: UserWarning: A learner ml_g has been provided for score = "partialling out" but will be ignored. "A learner ml_g is not required for estimation.
  warnings.warn(('A learner ml_g has been provided for score = "partialling out" but will be ignored. "'


In [28]:
table = PrettyTable()
table.field_names = ['Estimator', 'θ_hat', 'Std Error','t','p','2.5%','97.25%']
a = ['OLS']+ np.c_[OLS.params[1], OLS.bse[1], OLS.tvalues[1], OLS.pvalues[1], np.nan, np.nan].reshape(-1).tolist()
table.add_row(a)
a = ['IV-2SLS']+ np.c_[IV2SLS.params[1], IV2SLS.bse[1], IV2SLS.tvalues[1], IV2SLS.pvalues[1], np.nan, np.nan].reshape(-1).tolist()
table.add_row(a)
#a = ['Naive/Single ML (SML)']+ np.array(SML.summary).reshape(-1).tolist()
#table.add_row(a)
#a = ['Orthogonal ML (OML)']+ np.array(OML.summary).reshape(-1).tolist()
#table.add_row(a)
a = ['Orthogonal+Crossfitting ML (DML)']+ np.array(DML.summary).reshape(-1).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+----------------------------------+-------+-----------+--------+-------+-------+--------+
|            Estimator             | θ_hat | Std Error |   t    |   p   |  2.5% | 97.25% |
+----------------------------------+-------+-----------+--------+-------+-------+--------+
|               OLS                | 0.048 |   0.003   | 18.122 | 0.000 |  nan  |  nan   |
|             IV-2SLS              | 0.147 |   0.038   | 3.908  | 0.000 |  nan  |  nan   |
| Orthogonal+Crossfitting ML (DML) | 0.166 |   0.054   | 3.061  | 0.002 | 0.060 | 0.272  |
+----------------------------------+-------+-----------+--------+-------+-------+--------+
